In [16]:
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import metrics
import matplotlib.pyplot as plt

In [17]:
data_kata = pd.read_csv('D:\zzKoding\Data Sains - Machine Learning - AI\Lomba\Satria Data 2020\Data\data_kata_csv.csv')
data_kata_lema = [x for x in data_kata.lema]
words = list(data_kata.lema)

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

import itertools
dict(itertools.islice(WORDS.items(), 10))

import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [18]:
!pip install Sastrawi
# stopwords removal
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop = StopWordRemoverFactory().get_stop_words()

#stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [19]:
sentence = 'ditendang'
print(sentence)
sentence = stemmer.stem(sentence)
print(sentence)

ditendang
tendang


In [20]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [21]:
from nltk.corpus import stopwords

# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'lg', 'deh', 'toh', 'piye', 'prettt', 
                       'to', 'of', 'tak', 'sblm', 'dlm', 'sekedar'])
stop_words = list_stopwords

In [22]:
add_stopw = list()
def HiSed(df, perintah, stop_words, lema, correction, stemmer, add_stopw):
    iii = 0
    if perintah == 'sederhana':
        df_return = pd.DataFrame()
    elif perintah == 'hitung':
        dfCounts_return = dict()
        
    for index, row in df.iterrows():
        filter_sentence = ''
        sentence = row['total']
        # Cleaning dengan regular expression
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        # Tokenization
        words = nltk.word_tokenize(sentence)
        # Menghilangkan stopwords
        words = [w for w in words if not w in stop_words]
        print(iii)
        print(words)
        iii += 1
        if perintah == 'sederhana':
            # Lower case
            for words in words:
                kata_temp = words.lower()
                if kata_temp in lema:
                    filter_sentence = filter_sentence  + ' ' + kata_temp
                else:
                    cor_kat = correction(kata_temp)
                    if cor_kat != kata_temp:
                        filter_sentence = filter_sentence  + ' ' + cor_kat
                    else:
                        ste_kat = stemmer.stem(kata_temp)
                        if ste_kat != kata_temp:
                            filter_sentence = filter_sentence  + ' ' + ste_kat
                        else:
                            if kata_temp not in add_stopw:
                                add_stopw.append(kata_temp)
            df_return.loc[index, 'total'] = filter_sentence
        elif perintah == 'hitung':
            for word in words:
                if word.lower() in dfCounts_return:
                    dfCounts_return[word.lower()] += 1
                else:
                    dfCounts_return[word.lower()] = 1
            
    if perintah == 'sederhana':
        return df_return, add_stopw
    elif perintah == 'hitung':
        return dfCounts_return

In [23]:
df = pd.read_excel('D:\zzKoding\Data Sains - Machine Learning - AI\Lomba\Satria Data 2020\Data\Salinan Data Latih BDC.xlsx')
df_judul = df['judul']
df_narasi = df['narasi']
df_label = df['label']
df.head(10)

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg
5,736,1,2020-06-17 00:00:00,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma...",736.png
6,1161,1,2020-05-05 00:00:00,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...,1161.png
7,1597,1,2020-03-24 00:00:00,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...,1597.png
8,2098,1,2020-01-09 00:00:00,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...,2098.jpg
9,2226,1,2019-12-24 00:00:00,Video Polisi china telah menganiaya wanita uig...,Polisi china telah menganiaya wanita uighur le...,2226.png


In [24]:
df.shape
df = df.drop(['tanggal', 'nama file gambar'], axis=1)
df.head(10)

,ID,label,judul,narasi
0,71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...
1,461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
2,495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
3,550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
4,681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .
5,736,1,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma..."
6,1161,1,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UN...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...
7,1597,1,Foto Presiden Italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...
8,2098,1,"Kapolres Timor Tengah Utara , Nusa Tenggara Ti...",Polisi goblok. Bukanya cpet” d tolongin malah ...
9,2226,1,Video Polisi china telah menganiaya wanita uig...,Polisi china telah menganiaya wanita uighur le...


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4231 entries, 0 to 4230
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      4231 non-null   int64 
 1   label   4231 non-null   int64 
 2   judul   4231 non-null   object
 3   narasi  4231 non-null   object
dtypes: int64(2), object(2)
memory usage: 132.3+ KB


In [26]:
df['label'].value_counts()

1    3465
0     766
Name: label, dtype: int64

In [27]:
df['total']=df['judul'] + ' ' + df['narasi']
df.head()

,ID,label,judul,narasi,total
0,71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,Pemakaian Masker Menyebabkan Penyakit Legionna...
1,461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,Instruksi Gubernur Jateng tentang penilangan ...
2,495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,Foto Jim Rohn: Jokowi adalah presiden terbaik ...
3,550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...","ini bukan politik, tapi kenyataan Pak Jokowi b..."
4,681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,Foto Kadrun kalo lihat foto ini panas dingin K...


In [28]:
df_t = pd.read_excel('D:\zzKoding\Data Sains - Machine Learning - AI\Lomba\Satria Data 2020\Data\Data Uji BDC.xlsx')
df_t_judul = df_t['judul']
df_t_narasi = df_t['narasi']
df_t_label = df_t['label']
df_t_id = df_t['ID']
df_t.head()

,ID,tanggal,judul,narasi,nama file gambar,label
0,238057,2020-07-13 00:00:00,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,238057.jpg,1
1,238158,2020-07-06 00:00:00,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,238158.jpg,1
2,238865,2020-04-22 00:00:00,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,238865.jpg,1
3,248298,2019-10-22 00:00:00,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,248298.jpg,1
4,255176,2020-05-01 00:00:00,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",255176.jpg,1


In [29]:
df_t['total']=df_t['judul'] + ' ' + df_t['narasi']
df_t = df_t.drop(['tanggal', 'nama file gambar'], axis=1)
df_t.head()

,ID,judul,narasi,label,total
0,238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,Narasi Tito Karnavian Berideologi Komunis Kare...
1,238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,Anies: Seberat beratnya Pekerjaan Akan terasa ...
2,238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,Hindu di india Melemparkan Patung Buatan Merek...
3,248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...
4,255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,Permohonan Kelonggaran Angsuran ke OJK Untuk s...


In [30]:
total_temp, add_stopw = HiSed(df, 'sederhana', stop_words, data_kata_lema, correction, stemmer, add_stopw)

0
['Pemakaian', 'Masker', 'Menyebabkan', 'Penyakit', 'Legionnaires', 'A', 'caller', 'a', 'radio', 'talk', 'show', 'recently', 'shared', 'that', 'his', 'wife', 'was', 'hospitalized', 'told', 'she', 'had', 'COVID', 'only', 'a', 'couple', 'days', 'left', 'live', 'A', 'doctor', 'friend', 'suggested', 'she', 'be', 'tested', 'for', 'legionnaires', 'disease', 'because', 'she', 'wore', 'the', 'same', 'mask', 'every', 'day', 'all', 'day', 'long', 'Turns', 'out', 'it', 'WAS', 'legionnaires', 'disease', 'from', 'the', 'moisture', 'bacteria', 'in', 'her', 'mask', 'She', 'was', 'given', 'antibiotics', 'within', 'two', 'days', 'was', 'better', 'WHAT', 'IF', 'these', 'spikes', 'in', 'COVID', 'are', 'really', 'something', 'else', 'due', 'mask', 'induced', 'infections']
1
['Instruksi', 'Gubernur', 'Jateng', 'penilangan', 'bermasker', 'muka', 'Rp', 'etilang', 'Via', 'apps', 'PIKOBAR', 'YthSeluruh', 'Anggota', 'Grup', 'Sesuai', 'Instruksi', 'Gubernur', 'Jawa', 'Tengah', 'Hasil', 'Rapat', 'Tim', 'Gugus', 

25
['Corona', 'hoaks', 'CORONA', 'IS', 'A', 'HOAX']
26
['Kapal', 'Kargo', 'Pengangkut', 'Kadrun', 'tenggelam', 'Laut', 'China', 'Selatan', 'Kapal', 'Kargo', 'Indonesia', 'tujuan', 'China', 'dilaporkan', 'tenggelam', 'Laut', 'China', 'Selatan', 'Sabtu', 'malam', 'akibat', 'kebocoran', 'lambung', 'Kapal', 'Kapal', 'mengangkut', 'Kadrun', 'tujuan', 'China', 'tujuan', 'objek', 'percobaan', 'Vaksin', 'Covid', 'dimana', 'harga', 'monyet', 'China', 'mahal', 'harga', 'Kadrun', 'kali', 'murah', 'harga', 'monyet', 'Sampai', 'penyelidikan', 'Otoritas', 'Tiongkok', 'kecelakaan', 'fatal']
27
['Foto', 'Gubernur', 'Anies', 'Makan', 'Sekumpulan', 'Orang', 'Jawa', 'Tulen', 'Terlihat', 'sekumpulan', 'orang', 'Jawa', 'tulen', 'menyantap', 'kuliner', 'khas', 'Jawa', 'nasi', 'samin', 'kebab', 'buaya', 'panggang', 'minumannya', 'fifis', 'onta', 'segar', 'Percakapan', 'memakai', 'bahasa', 'Jawa', 'halus', 'alias', 'kromo', 'inggil', 'pulaSeperti', 'ana', 'akhi', 'antum', 'afwan', 'Hmmm', 'sungguh', 'mengagum

58
['Kronologis', 'Polisi', 'Berondong', 'Sedan', 'Terobos', 'Razia', 'Lubuklinggau', 'Innalillah', 'Hanya', 'Karena', 'Lalai', 'Terobos', 'Razia', 'Mobil', 'Berisi', 'Satu', 'Keluarga', 'Ditembaki', 'Polisi', 'Satu', 'Orang', 'Tewas']
59
['Kemristekdikti', 'undang', 'dosen', 'asing', 'profesor', 'kelas', 'dunia', 'Bukan', 'dosen', 'asing', 'profesor', 'kelas', 'dunia', 'Orang', 'Indonesia', 'mengikuti', 'program', 'mengajar', 'universitas', 'negeri', 'koneksi', 'peraih', 'Nobel', 'Direktur', 'Jenderal', 'Sumber', 'Daya', 'Iptek', 'Dikti', 'Kemristekdikti', 'Ali', 'Ghufron', 'Mukti', 'Jakarta', 'Selasa']
60
['Kecam', 'Pernyataan', 'Said', 'Aqil', 'Neno', 'Warisman', 'Kami', 'Para', 'Ibu', 'Tidak', 'Terima', 'Kecam', 'Pernyataan', 'Said', 'Aqil', 'Neno', 'Warisman', 'Kami', 'Para', 'Ibu', 'Tidak', 'Terima', 'Pernyataan', 'kontroversial', 'Ketua', 'PBNU', 'Said', 'Aqil', 'Siradj', 'nonton', 'film', 'porno', 'dengarkan', 'ceramah', 'radikal', 'istighfar', 'kecaman', 'pegiat', 'pendidikan'

84
['AM', 'HendroPriyono', 'Meninggal', 'Di', 'Singapura', 'INNALILAHIWAINALRAJIUN', 'TELAH', 'MATI', 'SEORANG', 'PEMBANTAI', 'RAKYAT', 'HENDROPRAYONO', 'D', 'SINGAPOREJGN', 'D', 'UCAPI', 'MENINGGALKARNA', 'DIA', 'PEMBANTAI', 'MANUSIA']
85
['Penjelasan', 'terkait', 'Video', 'Truk', 'Kontainer', 'Bergambar', 'JokowiMaruf', 'Amin', 'Truk', 'semi', 'kontainer', 'Paslon', 'kirakira', 'membawa']
86
['Bahaya', 'SemutKutu', 'Jepang', 'DIABETES', 'INFO', 'Bahaya', 'SemutKutu', 'Jepang', 'DIABETES', 'Ada', 'sahabat', 'sy', 'Magelang', 'pengidap', 'Diabetes', 'kmd', 'konsumsi', 'SemutKutu', 'Jepang', 'mengobati', 'Diabetes', 'Semut', 'mengeluarkan', 'insulin', 'Tapi', 'SATU', 'TAHUN', 'mengkonsumsi', 'SemutKutu', 'Jepang', 'ususnya', 'terkena', 'bakteri', 'rusak', 'dipotong', 'ususnya']
87
['CNN', 'Indonesia', 'Buat', 'Berita', 'Berjudul', 'SERTIFIKAT', 'GANDA', 'KERJA', 'REZIM', 'JOKOWI', 'CNN', 'INDONESIA', 'SERTIFIKAT', 'GANDA', 'KERJA', 'REZIM', 'JOKOWI', 'November']
88
['paham', 'politik', 

114
['Sejumlah', 'Polisi', 'India', 'Telah', 'Tiba', 'Di', 'Indonesia', 'Kabar', 'Terkini', 'Sejumlah', 'Polisi', 'India', 'Telah', 'Tiba', 'Di', 'Indonesia', 'Untuk', 'menertibkan', 'Masyarakat', 'tinggal', 'rumahSiapkan', 'Pantat', 'Minyak', 'Gosok']
115
['Pasukan', 'Tentara', 'Membantu', 'Melawan', 'Virus', 'Corona', 'New', 'York', 'Military', 'vehicles', 'arrived', 'in', 'ManhattanNEW', 'YORK', 'WUHAN', 'CORONAVIRUS']
116
['Video', 'KELUYURAN', 'DILUAR', 'RUMAH', 'DI', 'MALAYSIA', 'DITEMBAK', 'API', 'PAKAI', 'DRONE', 'KELUYURAN', 'DILUAR', 'RUMAH', 'DI', 'MALAYSIA', 'DITEMBAK', 'API', 'PAKAI', 'DRONE']
117
['Antisipasi', 'Corona', 'Seluruh', 'BUMN', 'DKI', 'Ditutup', 'ANTISIPASI', 'CORONA', 'SELURUH', 'BUMN', 'DKI', 'DITUTUP']
118
['Jokowi', 'Halal', 'Dilengserkan', 'Manusia', 'gk', 'nilai', 'dn', 'kuwalitas', 'gk', 'mnjdi', 'apapun', 'negeri']
119
['Sandiwara', 'Dagelan', 'koplax', 'perskusi', 'sendiridi', 'viralkan', 'sendiridi', 'laporkan', 'maaf', 'memaafkan', 'Sandiwara', 'Dag

...

4076
['Pemberitaan', 'Gerakan', 'Buruh', 'Jakarta', 'Dukung', 'AhokDjarot', 'Di', 'Portal', 'Bloombergindonesiamy', 'Pada', 'portal', 'daring', 'diberitakan', 'Gerakan', 'Buruh', 'Jakarta', 'GBJ', 'mendeklarasikan', 'mendukung', 'pasangan', 'CagubCawagub', 'Basuki', 'Tjahaja', 'Purnama', 'Djarot', 'Saiful', 'Hidayat']
4077
['Marinir', 'AS', 'Mendukung', 'Papua', 'Lepas', 'NKRI', 'Belum', 'foto', 'Marinir', 'AS', 'mendukung', 'Papua', 'lepas', 'NKRI', 'sayangnya', 'warga', 'BERLAGAK', 'SOK', 'Singa', 'Tapi', 'sadar', 'Si', 'Singa', 'mudah', 'dikalahkan', 'Kancil', 'Strategi', 'jitu']
4078
['Radio', 'Elshinta', 'Kasus', 'Prank', 'Daging', 'Kurban', 'Isi', 'Sampah', 'Bersama', 'Aribowo', 'Sasmito', 'CoFounder', 'Ketua', 'Komite', 'Pemeriksa', 'Fakta', 'MAFINDO']
4079
['zodiak', 'NASA', 'Ophiuchus', 'November', 'Desember', 'zodiak', 'NASA']
4080
['Pasien', 'Covid', 'Klaster', 'Hajatan', 'Boyolali', 'Dijemput', 'Petugas', 'Biaya', 'Dibebankan', 'Pemilik', 'Hajatan', 'Penjemputan', 'warga', 

4111
['Permen', 'Dot', 'Narkoba', 'Permen', 'Dot', 'Narkoba']
4112
['Buku', 'Berkonten', 'Sekolah', 'Komunisme', 'Konten', 'muncul', 'buku', 'kisikisi', 'mata', 'pelajaran', 'Bahasa', 'Inggris', 'halaman', 'judul', 'New', 'Mentor', 'Rahasia', 'Cerdas', 'Membedah', 'Kisikisi', 'UN', 'Bahasa', 'Inggris', 'SMAMA', 'tebal', 'halaman', 'Dalam', 'latihan', 'dimuat', 'teks', 'berjudul', 'Announcing', 'The', 'Robert', 'S', 'Kenny', 'Prize', 'gambar', 'palu', 'arit']
4113
['Surat', 'Terbuka', 'KM', 'ITB', 'Beredar', 'surat', 'terbuka', 'judul', 'Pemerintah', 'Berhenti', 'Serampangan', 'Kelola', 'negara', 'mengatasnamakan', 'Keluarga', 'Mahasiswa', 'ITB', 'Surat', 'mengkritisi', 'kebijakan', 'Presiden', 'Republik', 'Indonesia', 'Joko', 'Widodo', 'Ada', 'poin', 'dikritisi', 'surat', 'terbuka', 'kenaikan', 'bahan', 'bakar', 'minyak', 'BBM', 'pencabutan', 'subsidi', 'listrik', 'VA', 'hilirisasi', 'minerbapaska', 'rencana', 'perubahan', 'keempat', 'PP', 'kenaikan', 'harga', 'STNK', 'TNBK', 'BPKB', '

4143
['Penyemprotan', 'Disinfektan', 'DKI', 'Jakarta', 'Dimulai', 'Maret', 'Warga', 'Dihimbau', 'Tak', 'Melintas', 'Untuk', 'warga', 'DKI', 'JAKARTA', 'Pada', 'tanggal', 'Maret', 'Pukul', 'sd', 'wib', 'dilarang', 'berpergian', 'rumah', 'kendaraan', 'roda', 'berjalan', 'kaki', 'Dikarenakan', 'pencegahan', 'COVID', 'penyemprotan', 'Disinfektan', 'dosis', 'tingkat', 'udara', 'alergi', 'kulit', 'gangguan', 'pernafasan', 'Sebagai', 'langkah', 'Pemerintah', 'kepolisian', 'lintas', 'untung', 'pengaturan', 'kendaraan']
4144
['Korban', 'penculikan', 'dijahit', 'perutnya', 'Korban', 'penculikan', 'tenggelam', 'sungai', 'goyang', 'badannya', 'dibuka', 'bajunya', 'dijahit', 'perutnya']
4145
['Gadis', 'Ular', 'Gemparkan', 'Thailand', 'Gadis', 'berusia', 'Mai', 'Li', 'Fay', 'Bangkok', 'kehidupan', 'gadis', 'sebayanya']
4146
['Ini', 'sumbernya', 'Din', 'Syamsudin', 'Presiden', 'Joko', 'Widodo', 'menolak', 'menemui', 'ulama', 'Uzbekistan', 'Xinjiang', 'Cina', 'Delegasi', 'muslim', 'Uighur', 'diantar',

4176
['Info', 'Rekrutmen', 'CPNS', 'dibuka', 'Tanggal', 'Oktober', 'Resmi', 'buka', 'pendaftaran', 'CPNS', 'PPPK', 'PK', 'Pemerintah', 'membuka', 'penerimaan', 'CPNS', 'PPPK', 'PK', 'tahap', 'duaTahun', 'sebayak', 'Tenaga', 'Guru', 'SDSMP', 'SMUSMK', 'orang', 'Tenaga', 'Kesehatan', 'Dokter', 'Bidan', 'Perawat', 'dll', 'orang', 'Tenaga']
4177
['Tukar', 'menukar', 'aset', 'negara', 'PT', 'KCIC', 'Kodam', 'III', 'Siliwangi', 'Ternyata', 'surat', 'beredar', 'tukar', 'menukar', 'TNI', 'kereta', 'cepat', 'HOAX']
4178
['Seorang', 'pria', 'membintangi', 'film', 'berperan', 'Nabi', 'Muhammad', 'mati', 'terbakar', 'Seorang', 'pria', 'membintangi', 'film', 'berperan', 'sbg', 'Nabi', 'Muhammad', 'mati', 'terbakar', 'Amerika', 'menyembunyikan', 'berita', 'Sebarkanlah', 'kabar', 'Ada', 'wanita', 'Palestina', 'menyebarkan', 'kabar', 'namanya', 'Muhammad', 'lantas', 'bersumpah', 'jam', 'bermimpi', 'ketemu', 'Nabi', 'Muhammad', 'SAW', 'bersabda', 'Telah', 'kabar', 'kpd', 'kaum', 'muslimin', 'menyebarka

4207
['Video', 'Sudah', 'Lockdown', 'Sikafir', 'China', 'Terus', 'Masuk', 'Ke', 'Indonesia', 'Di', 'Seluruh', 'Negara', 'Lain', 'Sudah', 'Lockdown', 'Tapi', 'Sayang', 'Itu', 'Semua', 'Tidak', 'Berlaku', 'Negeriku', 'Katanya', 'Sikafir', 'China', 'Terus', 'Masuk', 'Keindonesia', 'Pak', 'Presiden', 'Tolong', 'Buka', 'Mata', 'Dong']
4208
['Foto', 'tweet', 'KPK', 'Anggota', 'dpr', 'dites', 'corona', 'positif', 'korupsi', 'anjing', 'Anggota', 'dpr', 'dites', 'corona', 'positif', 'korupsi', 'anjing']
4209
['Tes', 'Sederhana', 'Deteksi', 'Asam', 'Urat', 'Tes', 'Sederhana', 'Deteksi', 'Asam', 'Urat']
4210
['Video', 'Innalillahi', 'CommuterLine', 'emang', 'rawan', 'penularannya', 'Innalillahi', 'CommuterLine', 'emang', 'rawan', 'penularannya']
4211
['Passing', 'Grade', 'Universitas', 'Indonesia', 'Siapa', 'masuk', 'Universitas', 'Indonesia', 'UI', 'Kampus', 'impian', 'orang', 'BeritaBaik', 'SekolahNews', 'Daftar', 'Passing', 'Grade', 'Universitas', 'Indonesia', 'SMBPTN', 'httpssekolahnewscomdaf

In [31]:
print(add_stopw)

['legionnaires', 'recently', 'hospitalized', 'suggested', 'moisture', 'something', 'induced', 'infections', 'ythseluruh', 'indonesia', 'indonesiaruu', 'assistance', 'between', 'confederation', 'riswiss', 'terimakasih', 'nyinyirian', 'smartphone', 'semuanyacuman', 'adminnya', 'podomoro', 'moeldoko', 'pejuangpejuang', 'rohingya', 'banyumasan', 'fadlizon', 'httpstcojvjxcjhmp', 'depnaker', 'office', 'website', 'wwwdepnakeryolasitecom', 'whatsapp', 'hahhahhahahhaha', 'hermansyah', 'antimiras', 'mdagper', 'pulaseperti', 'sudjiatmi', 'yulianah', 'pelakupenculikan', 'dukcapil', 'prayforjapan', 'luaaaar', 'merahputih', 'tomorrow', 'yuuuuk', 'nantipertama', 'broadcast', 'gttptt', 'hercules', 'makassar', 'gantikhilafah', 'gantipresiden', 'netizentofa', 'zarazettirazr', 'roninkhalid', 'ndoroputrie', 'restycayah', 'ardiriau', 'sebaikbaik', 'lubuklinggau', 'innalillah', 'kemristekdikti', 'ghufron', 'banyumas', 'ngefly', 'purbalingga', 'benzodiazepin', 'nurmantyo', 'kopassus', 'coklatkenapa', 'muhama

In [32]:
display(df.head())
df['total'] = total_temp
display(df.head())

,ID,label,judul,narasi,total
0,71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,Pemakaian Masker Menyebabkan Penyakit Legionna...
1,461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,Instruksi Gubernur Jateng tentang penilangan ...
2,495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,Foto Jim Rohn: Jokowi adalah presiden terbaik ...
3,550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...","ini bukan politik, tapi kenyataan Pak Jokowi b..."
4,681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,Foto Kadrun kalo lihat foto ini panas dingin K...


,ID,label,judul,narasi,total
0,71,1,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,pemakaian masker menyebabkan penyakit ab cala...
1,461,1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,instruksi gubernur jayeng pengilangan masker ...
2,495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,foto jim roh joki presiden terbaik sejarah ba...
3,550,1,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",politik kenyataan pak joki berhasil memulangk...
4,681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,foto kadru lihat foto panas dingin kadru liha...


In [33]:
total_temp2, add_stopw = HiSed(df_t, 'sederhana', stop_words, data_kata_lema, correction, stemmer, add_stopw)

0
['Narasi', 'Tito', 'Karnavian', 'Berideologi', 'Komunis', 'Karena', 'Pernah', 'Disekolahkan', 'Partai', 'Komunis', 'China', 'Beijing', 'TITO', 'KARNIVAN', 'ITU', 'BERIDIOLOGI', 'KOMUNIS', 'DIA', 'BISA', 'DI', 'KATAKAN', 'PKI', 'KARENA', 'DI', 'PERNAH', 'DI', 'SEKOLAHLAH', 'OLEH', 'PARTAI', 'KOMUNIS', 'CHINA', 'DI', 'BAIJING']
1
['Anies', 'Seberat', 'beratnya', 'Pekerjaan', 'Akan', 'ringan', 'Bila', 'kerjakan', 'Seberat', 'beratnya', 'Pekerjaan', 'Akan', 'ringan', 'Bila', 'kerjakan']
2
['Hindu', 'india', 'Melemparkan', 'Patung', 'Buatan', 'Mereka', 'Ke', 'Laut', 'Karena', 'Tidak', 'Bisa', 'Menolong', 'Mereka', 'Dari', 'Corona', 'Hindu', 'india', 'melemparkan', 'patung', 'buatan', 'laut', 'lakukan', 'membela', 'virus', 'corona', 'Maha', 'Suci', 'Allah', 'Tuhan', 'Semesta', 'Alam']
3
['RSCM', 'Praktekkan', 'Penyedotan', 'Plug', 'VenaSaluran', 'Darah', 'Mulai', 'Hari', 'RSCM', 'diPraktekkan', 'Penyedotan', 'Plug', 'Sumbatan', 'VenaSaluran', 'Darah', 'Ke', 'Jantung', 'Tidak', 'Pake', 'Rin

32
['CengarCengir', 'Selfie', 'Tengah', 'Warga', 'Jakarta', 'Kebanjiran', 'Cengar', 'cengir', 'selfie', 'warga', 'Jakarta', 'kebanjiran']
33
['Masjid', 'Raya', 'DKI', 'bersimbol', 'Bintang', 'David', 'Yahudi', 'Masjid', 'Raya', 'DKI', 'bersimbol', 'Bintang', 'David', 'Yahudi']
34
['Sebuah', 'Akun', 'Facebook', 'Minta', 'Donasi', 'Untuk', 'Anak', 'Penderita', 'Kanker', 'Tulang', 'Bpk', 'presiden', 'RI', 'tolong', 'anak', 'kakak', 'mengidap', 'kanker', 'tulang', 'Tak', 'biaya', 'operasi', 'menahan', 'sakit', 'menangis', 'menghabiskan', 'biaya', 'berobat', 'kesana', 'kemari', 'kunjung', 'da', 'hasil', 'dokter', 'ambil', 'tindakan', 'bedah', 'paha', 'kanan', 'nyaTOLONG', 'orang', 'susah', 'pakk', 'orang', 'susah', 'rumah', 'sakit', 'melayani', 'Tolong', 'tolong', 'kasihan', 'kadang', 'malam', 'pahanya', 'gerogoti', 'didalam', 'menggigil', 'menahan', 'sakit']
35
['Nusron', 'Wahid', 'Siap', 'Diludahi', 'Jika', 'Peserta', 'Aksi', 'Lebih', 'Orang', 'Nusron', 'Wahid', 'Peserta', 'Aksi', 'Lebih'

65
['Liputan', 'Media', 'Acara', 'Diskusi', 'Publik', 'MAFINDO', 'Mitigasi', 'Hoaks', 'COVID', 'Mafindo', 'Hoax', 'Soal', 'Corona', 'Sama', 'Berbahaya', 'Virusnya']
66
['Kominfo', 'Bantah', 'Adanya', 'Kebocoran', 'Anggaran', 'Asian', 'Games', 'Sebesar', 'Rp', 'Juta', 'Asian', 'Games', 'Pernyataan', 'saudara', 'Jajang', 'Nurjaman', 'potensi', 'kerugian', 'negara', 'membandingkan', 'harga', 'penawaran', 'PT', 'IndoAd', 'PT', 'Bee', 'Work', 'Pariwara', 'relevan', 'Plt', 'Kepala', 'Biro', 'Humas', 'Kominfo', 'Noor', 'Iza', 'Selasa']
67
['Konvoi', 'Mobil', 'Militan', 'ISIS', 'Hujani', 'Bom', 'Oleh', 'Pesawat', 'Tempur', 'Iraq', 'Konvoi', 'Mobil', 'Militan', 'ISIS', 'Hujani', 'Bom', 'Oleh', 'Pesawat', 'Tempur', 'Iraq']
68
['Gambar', 'Judul', 'Artikel', 'Berjudul', 'Penyewa', 'VA', 'Adalah', 'Kader', 'PKS', 'KUALITAS', 'KADER', 'PARTAI', 'BERBASIS', 'ISLAM']
69
['ISTANA', 'TIDAK', 'ADA', 'LOCK', 'DOWN', 'DAERAH', 'KEPALA', 'DAERAH', 'YANG', 'MEMBUAT', 'ATURAN', 'SENDIRI', 'AKAN', 'DI', 'KENAK

99
['Ellen', 'DeGeneres', 'Membagikan', 'Bantuan', 'Untuk', 'Virus', 'Corona', 'Facebook', 'MY', 'NAME', 'ELLEN', 'DEGENERES', 'BECAUSE', 'BRONZE', 'CORONA', 'VIRUS', 'I', 'WANT', 'TO', 'HELP', 'FACEBOOK', 'FIRST', 'USER', 'MEMBERS', 'Tell', 'me', 'how', 'much', 'money', 'do', 'you', 'need', 'now', 'Choose', 'the', 'first', 'letter', 'your', 'name', 'and', 'you', 'will', 'receive', 'a', 'gift', 'your', 'choice']
100
['KORLAP', 'FPI', 'BANJIR', 'DI', 'JAKARTA', 'ADALAH', 'AIR', 'MATA', 'MALAIKAT', 'YANG', 'JATUH', 'SEBAB', 'MALAIKAT', 'TERHARU', 'PADA', 'ALUMNI', 'YANG', 'SERING', 'SHOLAT', 'MASSAL', 'DI', 'MONAS', 'KORLAP', 'FPI', 'BANJIR', 'DI', 'JAKARTA', 'ADALAH', 'AIR', 'MATA', 'MALAIKAT', 'YANG', 'JATUH', 'SEBAB', 'MALAIKAT', 'TERHARU', 'PADA', 'ALUMNI', 'YANG', 'SERING', 'SHOLAT', 'MASSAL', 'DI', 'MONAS']
101
['Polri', 'Luncurkan', 'Aplikasi', 'PolisiKu', 'Markas', 'Besar', 'Kepolisian', 'Republik', 'Indonesia', 'Divisi', 'Teknologi', 'Informasi', 'Div', 'TI', 'Polri', 'meluncurk

127
['Pasangan', 'Kakek', 'Nenek', 'Wafat', 'Mekkah', 'Saat', 'Haji', 'Innalillahi', 'wa', 'inna', 'illaihi', 'rojiun', 'meninggal', 'mekah', 'suami', 'istri', 'pakai', 'kursi', 'roda', 'didorong', 'pisah', 'subhanallah', 'terenyuh', 'hatiku', 'yaw', 'sungguh', 'kebesaran', 'Allah', 'SWT']
128
['Sekolah', 'SMAN', 'Rambah', 'Hilir', 'Riau', 'Tidak', 'Mewajibkan', 'Siswi', 'Non', 'Muslim', 'Memakai', 'Jilbab', 'SISWI', 'KRISTEN', 'WAJIB', 'PAKAI', 'JILBAB', 'DI', 'RIAU']
129
['Foto', 'ledakan', 'dahsyat', 'Beirut', 'gudang', 'penyimpanan', 'ton', 'amonium', 'nitrat', 'Terjadinya', 'ledakan', 'dahsyat', 'beirut', 'lokasi', 'kejadian', 'gudang', 'penyimpanan', 'ton', 'amonium', 'nitrat']
130
['Minum', 'Air', 'Es', 'Saat', 'Cuaca', 'Panas', 'Dapat', 'Mengalami', 'Pecah', 'Pembuluh', 'Darah', 'meminum', 'air', 'es', 'cuaca', 'panas', 'menyebabkan', 'pecahnya', 'pembuluh', 'darah', 'mikro']
131
['Foto', 'EKTP', 'milik', 'Warga', 'Cina', 'Cianjur', 'Bernama', 'Guohui', 'Chen', 'Disebut', 'Bisa

...

344
['BTW', 'Listrik', 'diem', 'tu', 'cebong', 'ngeluh', 'gk', 'BTW', 'Listrik', 'diem', 'tu', 'cebong', 'ngeluh', 'gk']
345
['homo', 'lesbi', 'dilindungi', 'hukum', 'UU', 'Maka', 'heran', 'Indonesia', 'mayoritasnya', 'Muslim', 'Tapi', 'kelompok', 'homo', 'lesbi', 'dilindungi', 'hukum', 'UU']
346
['Video', 'Berjudul', 'Melayu', 'Singapura', 'Negeri', 'Kami', 'Diambil', 'Orang', 'Melayu', 'Singapura', 'Negeri', 'Kami', 'Diambil', 'Orang', 'Dulu', 'Singapura', 'kota', 'sederhana', 'muslim', 'bebas', 'bahagia', 'Sekarang', 'suara', 'mengaji', 'suara', 'adzan', 'kecuali', 'masjid', 'Jumlah', 'masjid', 'Di', 'kota', 'mallmall', 'sedia', 'sholat', 'Dulu', 'bebas', 'makan', 'suka', 'Sekarang', 'makanan', 'halal', 'susah', 'dicari', 'kecuali', 'kampungkampung', 'Ke', 'kota', 'bawa', 'bekal', 'mestilah', 'puasa', 'makanan', 'lihat', 'halal', 'Bahasa', 'Melayu', 'dipakai', 'Semua', 'orang', 'cakap', 'Inggris', 'Mandarin', 'Negeri', 'diambil', 'orang']
347
['Video', 'orang', 'Amerika', 'teriak', 

369
['Jus', 'Jahe', 'Lada', 'Hitam', 'Mampu', 'Sembuhkan', 'Covid', 'Pada', 'mahasiswa', 'INDIAN', 'universitas', 'PONDICHERRY', 'bernama', 'RAMU', 'menemukan', 'obat', 'rumahan', 'Covid', 'kalinya', 'diterima', 'WHO']
370
['Klarifikasi', 'SMS', 'Dengan', 'Nomor', 'Handphone', 'Ustadz', 'Abdul', 'Somad', 'Dukung', 'Paslon', 'Bismillahirrahmanirrahim', 'UAS', 'dukungan', 'paslon', 'dasar', 'pertimbangan', 'umat', 'masyarakat', 'Riau']
371
['Paspor', 'Indonesia', 'Baru', 'Ada', 'Berhologram', 'Bintang', 'Bukan', 'Garuda', 'Lagi', 'Paspor', 'Indonesia', 'berhologram', 'bintang', 'Garuda', 'tunda', 'perpanjang', 'paspornya', 'gila', 'rezim']
372
['Joe', 'Ramadhan', 'menghina', 'Alm', 'Mbah', 'Moen', 'reporter', 'tvOne', 'Saya', 'tegaskan', 'nama', 'Joe', 'Ramadhan', 'tercatat', 'karyawan', 'jaringan', 'wartawan', 'kontributor', 'tvOne', 'Indonesia', 'negeri']
373
['Demo', 'Tolak', 'TKA', 'CINA', 'Solok', 'Diawali', 'Dengan', 'Pembakaran', 'Perahu', 'Warga', 'Oleh', 'Aparat', 'PERTANDA', 'G

401
['Why', 'does', 'Canada', 'refuse', 'eliminate', 'pork', 'from', 'the', 'school', 'diet', 'Why', 'does', 'Canada', 'refuse', 'eliminate', 'pork', 'from', 'the', 'school', 'diet', 'Many', 'Muslims', 'in', 'Canada', 'ask', 'that', 'pigs', 'not', 'be', 'served', 'in', 'the', 'cafeteria', 'canteen', 'and', 'restaurants', 'in', 'Montreal', 'Motreal', 'governors', 'answer', 'like', 'this', 'Finally', 'they', 'Muslims', 'in', 'Canada', 'must', 'understand', 'that', 'in', 'Canada', 'Quebec', 'with', 'its', 'JudeoChristian', 'roots', 'Christmas', 'trees', 'churches', 'and', 'religious', 'festivals', 'religion', 'must', 'remain', 'on', 'private', 'territory', 'The', 'Municipality', 'Dorval', 'is', 'right', 'reject', 'any', 'concessions', 'Islam', 'and', 'Sharia']
402
['REVOLUSI', 'MENTAL', 'PSI', 'REVOLUSI', 'MENTAL', 'PSI', 'by', 'Sepeda', 'man']
403
['Ini', 'RS', 'CANCER', 'SUMBER', 'WARAS', 'Basuki', 'Tjahaja', 'Purnama', 'Ini', 'RS', 'CANCER', 'SUMBER', 'WARAS', 'janjikan', 'Pak', 'Ahok'

433
['Tentara', 'Cina', 'Indonesia', 'Info', 'Penting', 'Tentara', 'Cina', 'Indonesia', 'Bersiaplah']
434
['Narasi', 'Kejadian', 'aneh', 'tp', 'nyata', 'video', 'potongan', 'daging', 'berkedut', 'Subhanallah', 'Kejadian', 'aneh', 'tp', 'nyata', 'didusun']
435
['Ahokers', 'Dituding', 'Mengubah', 'Data', 'KPU', 'Ahokers', 'Dituding', 'Mengubah', 'Data', 'KPU']
436
['Azab', 'Pada', 'Yahudi', 'Israel', 'Sebuah', 'video', 'diunggah', 'tanggal', 'Juli', 'akun', 'nama', 'Meli', 'Abdillah', 'foto', 'SS', 'WA', 'berisi', 'tulisan', 'Sementara', 'rakyat', 'Palestina', 'menderita', 'diserbu', 'tentara', 'Israel', 'mempertahankan', 'Al', 'Aqso', 'rakyat', 'Israel', 'pesta', 'pora', 'bar', 'Tel', 'Aviv', 'Ketika', 'berdansa', 'tibatiba', 'tanah', 'terbuka', 'orangorang', 'tertelan', 'masuk', 'ALLAH', 'SUDAH', 'MURKA', 'DENGAN', 'KAUM', 'YAHUDI', 'Saat', 'dicapture', 'postingan', 'dibagikan', 'kali']
437
['Warsito', 'dilarang', 'Riset', 'kanker', 'Pemerintah', 'Dipaksa', 'hentikan', 'risetnya', 'Dr'

468
['Klarifikasi', 'KPK', 'Terkait', 'Foto', 'Setya', 'Novanto', 'Tanpa', 'Baju', 'Tahanan', 'Secara', 'sengaja', 'WIB', 'rest', 'area', 'kilometer', 'Tol', 'Purbaleunyi', 'arah', 'Jakarta', 'rombongan', 'Investigasi', 'Gabungnya', 'Wartawan', 'Indonesia', 'GWI', 'orang', 'pengawalan', 'mantan', 'Ketua', 'DPRRI', 'Setya', 'Novanto', 'Setnov', 'terpidana', 'penjara']
469
['Erdogan', 'Tidak', 'Pernah', 'Akui', 'Kemenangan', 'Jokowi', 'Artikel', 'laman', 'daring', 'aljazeraonline', 'berjudul', 'Info', 'Dari', 'Turki', 'Erdogan', 'Tidak', 'Pernah', 'Akui', 'Kemenangan', 'Jokowi', 'memberitakan', 'penolakan', 'Recep', 'Tayyip', 'Erdogan', 'Presiden', 'Turki', 'kemenangan', 'Calon', 'Presiden', 'Nomor', 'Urut', 'Joko', 'Widodo', 'pasca', 'kemenangan', 'Jokowi', 'berdasarkan', 'hasil', 'hitung', 'cepat', 'lembaga', 'survei']


In [34]:
display(df_t.head())
df_t['total'] = total_temp2
display(df_t.head())

,ID,judul,narasi,label,total
0,238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,Narasi Tito Karnavian Berideologi Komunis Kare...
1,238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,Anies: Seberat beratnya Pekerjaan Akan terasa ...
2,238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,Hindu di india Melemparkan Patung Buatan Merek...
3,248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...
4,255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,Permohonan Kelonggaran Angsuran ke OJK Untuk s...


,ID,judul,narasi,label,total
0,238057,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,1,narasi dito karavan berideologi komunis karen...
1,238158,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,1,anis sejerat bertanya pekerjaan akan ringan b...
2,238865,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,1,bindu indik melemparkan patung buatan mereka ...
3,248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,1,asam praktikan penyedotan plug darah mulai ha...
4,255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",1,permohonan kelonggaran angsuran jok untuk inf...


In [35]:
list_stopwords.extend(add_stopw)
stop_words = list_stopwords

In [36]:
#inisiasi x dan y
x_train = df['total']
y_train = df['label']

In [37]:
x_test = df_t['total']
y_test = df_t['label']

In [38]:
# Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(max_df=1.0)

#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

In [39]:
tfidf_train_array = tfidf_train.toarray()
tfidf_test_array = tfidf_test.toarray()
tfidf_train_df = pd.DataFrame(tfidf_train_array)
tfidf_test_df = pd.DataFrame(tfidf_test_array)
display(tfidf_train_df.head())
display(tfidf_test_df.head())

,0,1,2,3,4,5,6,7,8,9,...,11214,11215,11216,11217,11218,11219,11220,11221,11222,11223
0,0.208258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.104175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,11214,11215,11216,11217,11218,11219,11220,11221,11222,11223
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(tfidf_train_df, y_train)
pred = model.predict(tfidf_test_df)
score = metrics.f1_score(y_test, pred)
print(score)

0.9453219927095989


In [41]:
def ParamSendiri(judul, narasi, label, stop_words):
    angka = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    tanda = ['?', '!', '.', ',', ' ']
    df_baru_return = pd.DataFrame()
    for index, row in judul.iteritems():
        kata_judul = row
        kata_narasi = narasi[index]
        
        if row[0] in angka:
            df_baru_return.loc[index, 'judul_awal_angka'] = 1
        elif row[0] not in angka:
            df_baru_return.loc[index, 'judul_awal_angka'] = 0
        if kata_narasi[0] in angka:
            df_baru_return.loc[index, 'narasi_awal_angka'] = 1
        elif kata_narasi[0] not in angka:
            df_baru_return.loc[index, 'narasi_awal_angka'] = 0
        
        temp = 0
        for x in kata_judul:
            if x in tanda:
                temp += 1
        jumlah_tanda_judul = temp
        df_baru_return.loc[index, 'judul_jumlah_tanda'] = jumlah_tanda_judul
        temp = 0
        for x in kata_narasi:
            if x in tanda:
                temp += 1
        jumlah_tanda_narasi = temp
        df_baru_return.loc[index, 'narasi_jumlah_tanda'] = jumlah_tanda_narasi
        
        kata_judul_bersih = re.sub(r'[^\w\s]', '', kata_judul)
        kata_judul_token = nltk.word_tokenize(kata_judul_bersih)
        jumlah_kata_judul = len(kata_judul_token)
        df_baru_return.loc[index, 'judul_jumlah_kata'] = jumlah_kata_judul
        if (jumlah_tanda_judul - jumlah_kata_judul) >= 0:
            df_baru_return.loc[index, 'judul_jumlah_tanda_tak_wajar'] = (jumlah_tanda_judul - jumlah_kata_judul)
        elif (jumlah_tanda_judul - jumlah_kata_judul) < 0:
            df_baru_return.loc[index, 'judul_jumlah_tanda_tak_wajar'] = 0
        kata_narasi_bersih = re.sub(r'[^\w\s]', '', kata_narasi)
        kata_narasi_token =  nltk.word_tokenize(kata_narasi_bersih)
        jumlah_kata_narasi = len(kata_narasi_token)
        df_baru_return.loc[index, 'narasi_jumlah_kata'] = jumlah_kata_narasi
        if (jumlah_tanda_narasi - jumlah_kata_narasi) >= 0:
            df_baru_return.loc[index, 'narasi_jumlah_tanda_tak_wajar'] = (jumlah_tanda_narasi - jumlah_kata_narasi)
        elif (jumlah_tanda_narasi - jumlah_kata_narasi) < 0:
            df_baru_return.loc[index, 'narasi_jumlah_tanda_tak_wajar'] = 0
            
        kata_judul_list = [w for w in kata_judul_token if not w in stop_words]
        temp2, temp3 = 0, 0
        for w in kata_judul_list:
            temp3 += 1
            if w.isupper() == True:
                temp2 += 1
        if temp2 == temp3:
            df_baru_return.loc[index, 'judul_kalimat_kapital'] = 1
        elif temp2 != temp3:
            df_baru_return.loc[index, 'judul_kalimat_kapital'] = 0
        kata_narasi_list = [w for w in kata_narasi_token if not w in stop_words]
        temp2, temp3 = 0, 0
        for w in kata_narasi_list:
            temp3 += 1
            if w.isupper() == True:
                temp2 += 1
        if temp2 == temp3:
            df_baru_return.loc[index, 'narasi_kalimat_kapital'] = 1
        elif temp2 != temp3:
            df_baru_return.loc[index, 'narasi_kalimat_kapital'] = 0

        # Menghilangkan stopwords
        kata_judul_notstopw = [w.lower() for w in kata_judul_token if not w in stop_words]
        kata_narasi_notstopw = [w.lower() for w in kata_narasi_token if not w in stop_words]
        temp4, temp5 = 0, 0
        for w in kata_narasi_notstopw:
            temp4 += 1
            if w in kata_judul_notstopw:
                temp5 += 1
        df_baru_return.loc[index, 'jumlah_keyword'] = temp5
        df_baru_return.loc[index, 'persentase_keyword'] = (temp5/temp4)*100

        df_baru_return.loc[index, 'label'] = label[index]
    return df_baru_return

In [42]:
df_baru_train = ParamSendiri(df_judul, df_narasi, df_label, stop_words)
df_baru_train.head(5)

,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda,narasi_jumlah_tanda,judul_jumlah_kata,judul_jumlah_tanda_tak_wajar,narasi_jumlah_kata,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,jumlah_keyword,persentase_keyword,label
0,0.0,0.0,4.0,99.0,5.0,0.0,88.0,11.0,0.0,0.0,0.0,0.000000,1.0
1,0.0,0.0,20.0,17.0,18.0,2.0,17.0,0.0,0.0,0.0,3.0,17.647059,1.0
2,0.0,0.0,10.0,14.0,11.0,0.0,13.0,1.0,0.0,0.0,9.0,81.818182,1.0
3,0.0,0.0,17.0,130.0,15.0,2.0,116.0,14.0,0.0,0.0,19.0,23.456790,1.0
4,0.0,0.0,7.0,10.0,8.0,0.0,7.0,3.0,0.0,0.0,5.0,100.000000,1.0


In [43]:
df_baru_train['label'] = df_baru_train['label'].astype(int)
display(len(df_baru_train[df_baru_train['judul_jumlah_tanda']>df_baru_train['judul_jumlah_kata']]))
display(len(df_baru_train[df_baru_train['judul_kalimat_kapital'] == 1]))
display(df_baru_train)

375

255

,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda,narasi_jumlah_tanda,judul_jumlah_kata,judul_jumlah_tanda_tak_wajar,narasi_jumlah_kata,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,jumlah_keyword,persentase_keyword,label
0,0.0,0.0,4.0,99.0,5.0,0.0,88.0,11.0,0.0,0.0,0.0,0.000000,1
1,0.0,0.0,20.0,17.0,18.0,2.0,17.0,0.0,0.0,0.0,3.0,17.647059,1
2,0.0,0.0,10.0,14.0,11.0,0.0,13.0,1.0,0.0,0.0,9.0,81.818182,1
3,0.0,0.0,17.0,130.0,15.0,2.0,116.0,14.0,0.0,0.0,19.0,23.456790,1
4,0.0,0.0,7.0,10.0,8.0,0.0,7.0,3.0,0.0,0.0,5.0,100.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226,0.0,0.0,8.0,7.0,8.0,0.0,8.0,0.0,0.0,0.0,8.0,100.000000,1
4227,0.0,0.0,5.0,28.0,6.0,0.0,28.0,0.0,0.0,0.0,5.0,23.809524,1
4228,0.0,0.0,8.0,5.0,9.0,0.0,6.0,0.0,0.0,0.0,1.0,25.000000,1
4229,0.0,0.0,12.0,12.0,12.0,0.0,11.0,1.0,0.0,0.0,10.0,100.000000,1


In [44]:
df_baru_test = ParamSendiri(df_t_judul, df_t_narasi, df_t_label, stop_words)
df_baru_test['label'] = df_baru_test['label'].astype(int)

In [45]:
display(df_baru_train.head(2))
display(df_baru_test.head(2))

df_baru_train = df_baru_train.drop(['judul_jumlah_tanda', 'narasi_jumlah_tanda', 
                                    'judul_jumlah_kata', 'narasi_jumlah_kata', 'jumlah_keyword'], axis=1)
df_baru_test = df_baru_test.drop(['judul_jumlah_tanda', 'narasi_jumlah_tanda',
                                  'judul_jumlah_kata', 'narasi_jumlah_kata', 'jumlah_keyword'], axis=1)

display(df_baru_train.head(2))
display(df_baru_test.head(2))

,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda,narasi_jumlah_tanda,judul_jumlah_kata,judul_jumlah_tanda_tak_wajar,narasi_jumlah_kata,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,jumlah_keyword,persentase_keyword,label
0,0.0,0.0,4.0,99.0,5.0,0.0,88.0,11.0,0.0,0.0,0.0,0.000000,1
1,0.0,0.0,20.0,17.0,18.0,2.0,17.0,0.0,0.0,0.0,3.0,17.647059,1


,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda,narasi_jumlah_tanda,judul_jumlah_kata,judul_jumlah_tanda_tak_wajar,narasi_jumlah_kata,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,jumlah_keyword,persentase_keyword,label
0,0.0,0.0,12.0,20.0,13.0,0.0,21.0,0.0,0.0,1.0,7.0,33.333333,1
1,0.0,0.0,10.0,9.0,11.0,0.0,10.0,0.0,0.0,0.0,7.0,100.000000,1


,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword,label
0,0.0,0.0,0.0,11.0,0.0,0.0,0.000000,1
1,0.0,0.0,2.0,0.0,0.0,0.0,17.647059,1


,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword,label
0,0.0,0.0,0.0,0.0,0.0,1.0,33.333333,1
1,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,1


In [46]:
x_train = df_baru_train.drop('label', axis=1)
y_train = df_baru_train['label']
x_test = df_baru_test.drop('label', axis=1)
y_test = df_baru_test['label']

In [47]:
display(tfidf_train_df.head(2))
display(x_train.head(2))
print('tfidf train:', len(tfidf_train_df))
print('x train:', len(x_train))
X_train = pd.concat([tfidf_train_df, x_train], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,11214,11215,11216,11217,11218,11219,11220,11221,11222,11223
0,0.208258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.104175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword
0,0.0,0.0,0.0,11.0,0.0,0.0,0.000000
1,0.0,0.0,2.0,0.0,0.0,0.0,17.647059


tfidf train: 4231
x train: 4231


In [48]:
display(X_train.head(2))

,0,1,2,3,4,5,6,7,8,9,...,11221,11222,11223,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword
0,0.208258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.000000
1,0.104175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,17.647059


In [49]:
display(tfidf_test_df.head(2))
display(x_test.head(2))
print('tfidf test:', len(tfidf_test_df))
print('x test:', len(x_test))
X_test = pd.concat([tfidf_test_df, x_test], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,11214,11215,11216,11217,11218,11219,11220,11221,11222,11223
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword
0,0.0,0.0,0.0,0.0,0.0,1.0,33.333333
1,0.0,0.0,0.0,0.0,0.0,0.0,100.000000


tfidf test: 470
x test: 470


In [50]:
display(X_train.head(2))
display(X_test.head(2))

,0,1,2,3,4,5,6,7,8,9,...,11221,11222,11223,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword
0,0.208258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.000000
1,0.104175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,17.647059


,0,1,2,3,4,5,6,7,8,9,...,11221,11222,11223,judul_awal_angka,narasi_awal_angka,judul_jumlah_tanda_tak_wajar,narasi_jumlah_tanda_tak_wajar,judul_kalimat_kapital,narasi_kalimat_kapital,persentase_keyword
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,33.333333
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.000000


In [51]:
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)
pred = model.predict(X_test)
score = metrics.f1_score(y_test, pred)
print(score)

0.9464720194647201


In [52]:
subset1_x, subset1_y = X_train, y_train

In [53]:
#PAC PAKE DF
from sklearn.metrics import f1_score
from sklearn.linear_model import PassiveAggressiveClassifier
# PASSIVE-AGGRESSIVE CLASSIFIER TF-IDF   

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(subset1_x,subset1_y)
#Predict on the test set
y_pred1 = pac.predict(X_test)

#F1 Score
scoref1 = f1_score(y_test, y_pred1)
print(f'F1 Score: {round(scoref1*100,2)}%')

F1 Score: 92.64%


In [54]:
subset2_x, subset2_y = X_train, y_train

In [55]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB(alpha=0.1)

# Fit to the training data
nb_classifier.fit(subset2_x, subset2_y)

# Predict the labels: pred
y_pred2 = nb_classifier.predict(X_test)

# Compute accuracy: score
score2 = metrics.f1_score(y_test, y_pred2)
print(f'F1 Score: {round(score2*100,2)}%')

F1 Score: 91.89%


In [56]:
subset3_x, subset3_y = X_train, y_train

In [57]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(subset3_x, subset3_y)
y_pred3 = logreg.predict(X_test)
score3 = metrics.f1_score(y_test, y_pred3)
print(f'F1 Score: {round(score3*100,2)}%')

C:\Users\ACER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


F1 Score: 92.09%


In [58]:
subset5_x, subset5_y = X_train, y_train

In [59]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(subset5_x, subset5_y)

#Predict the response for test dataset
y_pred5 = clf.predict(X_test)
score5 = metrics.f1_score(y_test, y_pred5)
print(f'F1 Score: {round(score5*100,2)}%')

F1 Score: 93.4%


In [60]:
from statistics import mode
final_pred = np.array([])
for i in range(0,len(tfidf_test_df)):
    final_pred = np.append(final_pred, mode([y_pred1[i], y_pred2[i], y_pred3[i], pred[i], y_pred5[i]]))

In [61]:
score_akhir = metrics.f1_score(y_test, final_pred)
print(f'F1 Score: {round(score_akhir*100,2)}%')

F1 Score: 92.75%
